<a href="https://colab.research.google.com/github/ishammansoor/GPT_Code/blob/main/Self_Attention_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import math


# L = the size of the input sequence

L, d_k, d_v = 4, 8, 8
q = np.random.randn(L, d_k)
k = np.random.randn(L, d_k)
v = np.random.randn(L, d_v)

In [3]:
print("q: ", q)
print("k: ", k)
print("v: ", v)

q:  [[ 1.18462419  1.71758272  0.30611785  0.29770678  0.02982607 -1.01316031
   0.09972832  0.71381426]
 [ 0.20305684  0.67294698  0.74518149  0.00385025  0.84204858 -0.09611828
   0.24116615  2.24492789]
 [-1.08131676  0.50745548 -1.1076831  -0.81451905  0.60821448 -2.63104095
  -0.09473142  0.46363733]
 [ 0.48983117 -1.49738923 -0.9149111   1.32269724 -0.33860081 -0.48114265
  -0.68322736 -2.83297443]]
k:  [[-0.41116239  1.23673511  1.82525303 -0.0743678  -0.70796471 -1.33160761
   0.81795132  0.70458952]
 [-0.23586027 -0.60761581  0.80330802 -0.7810145   0.53430326 -0.09057807
   0.35906457 -0.31456844]
 [ 2.40246523  1.44270587  0.48139606 -1.85387129  0.25706481 -0.08579798
   0.64726034 -0.16865336]
 [-0.78922179  1.8772332   0.50778735  1.66574773 -0.6577302   0.9845162
  -1.2754653   0.06326837]]
v:  [[ 0.44491195  0.07752544 -1.24250232  0.14289091  0.20318407  0.13991395
   0.09902659  0.60553795]
 [-0.75124429  0.74746088  0.99790688  0.83320647  1.16767153  0.34659134
   1

In [4]:
np.matmul(q, k.T)

array([[ 4.08625981, -1.39067085,  4.95819635,  1.84159085],
       [ 3.41949226, -0.02215683,  1.81248158,  0.67379192],
       [ 2.43306495,  0.0764674 , -0.64635904, -2.95342296],
       [-5.49610451, -0.46518843, -3.88623151, -1.01762421]])

In [5]:
q.var(), k.var(), np.matmul(q, k.T).var()

(1.1770021658090921, 0.9781453951049961, 7.6834159732854435)

In [7]:
scaled = np.matmul(q, k.T) / math.sqrt(d_k)
q.var(), k.var(), scaled.var()

(1.1770021658090921, 0.9781453951049961, 0.9604269966606803)

In [8]:
scaled

array([[ 1.44471101, -0.49167639,  1.75298713,  0.65110069],
       [ 1.20897308, -0.00783362,  0.64080901,  0.23822142],
       [ 0.86021836,  0.02703531, -0.22852243, -1.0441927 ],
       [-1.94316639, -0.16446895, -1.37399033, -0.35978449]])

In [10]:
mask = np.tril(np.ones((L, L)))
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [12]:
mask[mask == 0] = -np.infty
mask[mask == 1] = 0

In [13]:
mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [14]:
scaled + mask

array([[ 1.44471101,        -inf,        -inf,        -inf],
       [ 1.20897308, -0.00783362,        -inf,        -inf],
       [ 0.86021836,  0.02703531, -0.22852243,        -inf],
       [-1.94316639, -0.16446895, -1.37399033, -0.35978449]])

In [20]:
def softmax(x):
  return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T

def scaled_dot_product_attention(q, k, v, mask=None):
  d_k = q.shape[-1]
  scaled = np.matmul(q, k.T) / math.sqrt(d_k)
  if mask is not None:
    scaled = scaled + mask
  attention = softmax(scaled)
  out = np.matmul(attention, v)
  return out, attention

In [16]:
attention = softmax(scaled + mask)

In [17]:
attention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.7715011 , 0.2284989 , 0.        , 0.        ],
       [0.56455594, 0.24539188, 0.19005218, 0.        ],
       [0.07374441, 0.4367245 , 0.1302924 , 0.35923869]])

In [18]:
new_v = np.matmul(attention, v)
new_v

array([[ 0.44491195,  0.07752544, -1.24250232,  0.14289091,  0.20318407,
         0.13991395,  0.09902659,  0.60553795],
       [ 0.17159157,  0.23060495, -0.73057128,  0.30062726,  0.42356839,
         0.1871395 ,  0.52699839,  0.78775429],
       [-0.0208011 ,  0.26211622, -0.70282919,  0.39368699,  0.49308628,
        -0.03833245,  0.57043206,  1.06929113],
       [-0.16834502,  0.95568419,  0.12186948,  0.35089382,  0.51053344,
        -0.02285941,  0.22151483,  1.20031532]])

In [19]:
v

array([[ 0.44491195,  0.07752544, -1.24250232,  0.14289091,  0.20318407,
         0.13991395,  0.09902659,  0.60553795],
       [-0.75124429,  0.74746088,  0.99790688,  0.83320647,  1.16767153,
         0.34659134,  1.97199753,  1.40298747],
       [-0.46108149,  0.18378079, -1.29567237,  0.5711851 ,  0.48323778,
        -1.06482544,  0.16108124,  2.01602184],
       [ 0.52056593,  1.66905021, -0.14891784, -0.27264979, -0.2153533 ,
        -0.1275017 , -1.85947439,  0.78017472]])

In [ ]:
def scaled_dot_product_attention(q, k, v, mask=None):
  d_k = q.shape[-1]
  scaled = np.matmul(q, k.T) / math.sqrt(d_k)
  if mask is not None:
    scaled = scaled + mask
  attention = softmax(scaled)
  out = np.matmul(attention, v)
  return out, attention